# Loading

In [7]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm

ModuleNotFoundError: No module named 'gensim.summarization'

In [ ]:
datasets = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
        print(len(pieces))
        datasets.append(pieces)

In [ ]:
datasets[-1][-10]

In [ ]:
with open('../../dataset/English/articles/articles.json', 'r',encoding='utf-8') as f:
    raw_articles = json.load(f)
len(raw_articles)

In [ ]:
random.sample(raw_articles, 1)[0]

In [ ]:
corpus = [' '.join([a['title'], a['subtitle'], a['claim'], a['content']])
          for a in raw_articles]
len(corpus)

In [ ]:
random.sample(corpus, 1)[0]

In [ ]:
model = bm25.init_model(corpus)

## Example

In [ ]:
p = random.sample(random.sample(datasets, 1)[0], 1)[0]
print(p['content'], p['label'], p['time'])
print()

scores = bm25.get_a_query_scores(p['content'], model)
scores, scores.shape

for i, idx in enumerate(scores.argsort()[::-1][:10]):
    print('='*10, 'Top:',i, ', Score =', scores[idx], '='*10, '\n')
    print(raw_articles[idx]['date'])
    print(corpus[idx])
    print()

## Total

In [ ]:
for i, pieces in enumerate(datasets):
    queries = [p['content'] for p in pieces]
    scores_arr = bm25.get_total_scores(queries, corpus, bm25_model=model)
    np.save(
        'data/English/bm25_{}.npy'.format(['train', 'val', 'test'][i]), scores_arr)

# Export Top 10 (only articles published BEFORE the post)

## Loading

In [ ]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm
import numpy as np
import pickle

In [ ]:
datasets = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
        print(len(pieces))
        datasets.append(pieces)

In [ ]:
datasets[-1][-10]

In [ ]:
with open('../../dataset/Twitter/articles/Snopes_8084.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

In [ ]:
random.sample(raw_articles, 1)[0]

## Find the nearest article

In [ ]:
articles_times = [a['date'] for a in raw_articles]
len(articles_times)

In [ ]:
articles_times

In [ ]:
'2004-08-19 16:59:00' < '2005-08-12', '2004-08-19 16:59:00' > '1995-09-25'

In [ ]:
def find_the_nearest_article_idx(post_time):
    assert len(articles_times) == len(raw_articles)
    times = articles_times.copy()
    times.append(post_time)
    times.sort()
    return times.index(post_time) - 1

In [ ]:
p = random.sample(random.sample(datasets, 1)[0], 1)[0]
print(p['content'], '\n', p['label'], p['time'])
print()

idx = find_the_nearest_article_idx(p['time'])

print(articles_times[idx])
print()
print(articles_times[idx-5:idx+5])

## Export

### Example

In [ ]:
bm25_rank = np.load('./data/English/bm25_train.npy')
bm25_rank.shape

In [ ]:
bm25_rank

In [ ]:
ranked_arr = (-bm25_rank).argsort()

In [ ]:
ranked_arr

In [ ]:
bm25_rank[0][3399],bm25_rank[0][1210],bm25_rank[0][4978],bm25_rank[0][1033],

In [ ]:
datasets[0][0]

In [8]:
raw_articles[3399]

NameError: name 'raw_articles' is not defined

In [ ]:
idx = find_the_nearest_article_idx(datasets[0][0]['time'])
idx

In [ ]:
ranked_arr[0]

In [ ]:
valid_idxs = np.where(ranked_arr[0] <= idx, ranked_arr[0], -1)
valid_idxs = [i for i in valid_idxs.tolist() if i != -1]
valid_idxs[:10]

In [ ]:
raw_articles[1210]

### Total

In [ ]:
for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r') as f:
        pieces = json.load(f)
    print('Pieces: ', len(pieces))
    
    bm25_rank = np.load('./data/English/bm25_{}.npy'.format(t))
    print(t, bm25_rank.shape)
    
    print('\nSorting...')
    ranked_arr = (-bm25_rank).argsort()
    print('Done. ranked_arr: {}\n'.format(ranked_arr.shape))
    
    ranked_articles = []
    zero_num = 0
    for i, p in enumerate(tqdm(pieces)):
        idx = find_the_nearest_article_idx(p['time'])
        
        valid_idxs = np.where(ranked_arr[i] <= idx, ranked_arr[i], -1)
        valid_idxs = [aid for aid in valid_idxs.tolist() if aid != -1]
        if len(valid_idxs) == 0:
            zero_num += 1
        
        ranked_articles.append(valid_idxs)
        
    with open('./data/English/ranked_articles_{}.pkl'.format(t), 'wb') as f:
        pickle.dump(ranked_articles, f)
        
    print('Dataset: {}, zero_num = {}\n{}\n'.format(t, zero_num, '='*15))

### Top10

In [ ]:
import pickle
import random
import numpy as np
from tqdm import tqdm

In [ ]:
TOP=10

for t in ['train', 'val', 'test']:
    with open('./data/English/ranked_articles_{}.pkl'.format(t), 'rb') as f:
        ranked_articles = pickle.load(f)
    print('{}: {}'.format(t, len(ranked_articles)))
    
    # <Padding> value = -1
    top_arr = np.zeros((len(ranked_articles), TOP)) - 1
    for i, ranks in enumerate(tqdm(ranked_articles)):
        sz = min(TOP, len(ranks))
        top_arr[i, :sz] = ranks[:sz]
        
    file = './data/English/Top{}_articles_{}.npy'.format(TOP, t)
    np.save(file, top_arr)

In [ ]:
top_arr.shape

In [ ]:
top_arr

# Check

In [ ]:
import json
import random
import sys
import bm25
import numpy as np
from tqdm import tqdm

In [9]:
datasets = []
top_articles = []

for t in ['train', 'val', 'test']:
    with open('../../dataset/English/post/{}.json'.format(t), 'r',encoding='utf-8') as f:
        pieces = json.load(f)
    print(len(pieces))
    
    arr = np.load('./data/English/Top10_articles_{}.npy'.format(t))
    arr = np.array(arr, dtype=int)
    print(arr.shape)
    
    datasets.append(pieces)
    top_articles.append(arr)

UnicodeDecodeError: 'gbk' codec can't decode byte 0xa6 in position 378: illegal multibyte sequence

In [ ]:
with open('../../dataset/English/articles/articles.json', 'r') as f:
    raw_articles = json.load(f)
len(raw_articles)

In [ ]:
t = random.randint(0,2)
pieces = datasets[t]
arr = top_articles[t]

i = random.randint(0, len(pieces)-1)
p = pieces[i]
rank = arr[i]

print(p['content'], p['label'], p['time'])
print()

for i, idx in enumerate(rank):
    print('='*20, 'Top:',i, '='*20, '\n')
    print(raw_articles[idx]['date'])
    print(raw_articles[idx]['content'])
    print()